<a href="https://colab.research.google.com/github/kaushalprasadhial/face-recognition/blob/master/face_recognition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
!pwd

/content


In [0]:
path_to_zip_file = '/content/drive/My Drive/Colab Notebooks/face recognition/pins-face-recognition.zip'
directory_to_extract_to = '/content/drive/My Drive/Colab Notebooks/face recognition/pins-face-recognition/'
import zipfile
with zipfile.ZipFile(path_to_zip_file, 'r') as zip_ref:
    zip_ref.extractall(directory_to_extract_to)  

In [0]:
def movefile(source,dest):
    os.rename(source, dest)

In [0]:
import os
import shutil
temp_dir = '/content/drive/My Drive/Colab Notebooks/face recognition/images/temp/'
train_dir = '/content/drive/My Drive/Colab Notebooks/face recognition/images/train/'
test_dir = '/content/drive/My Drive/Colab Notebooks/face recognition/images/test/'


dir = next(os.walk(temp_dir))[1]
#print([temp_dir+path for path in dir],sep='\n')
#dir=['/content/drive/My Drive/Colab Notebooks/face recognition/images/temp/pins_Scarlett Johansson/','/content/drive/My Drive/Colab Notebooks/face recognition/images/temp/pins_Willa Holland','/content/drive/My Drive/Colab Notebooks/face recognition/images/temp/pins_alexandra daddario','/content/drive/My Drive/Colab Notebooks/face recognition/images/temp/pins_amber heard face','/content/drive/My Drive/Colab Notebooks/face recognition/images/temp/pins_anne hathaway']

for path in dir:
    onlyfiles = next(os.walk(temp_dir+path))[2] #dir is your directory path as string
    train_files = onlyfiles[:int(len(onlyfiles)*.8)]
    test_files = onlyfiles[int(len(onlyfiles)*.8):]
    os.mkdir(train_dir+path+'/')
    os.mkdir(test_dir+path+'/')
    for file in train_files: 
        source_path = temp_dir+path+'/'+file
        
        dest_path = train_dir+path+'/'+file
#         print(source_path,'\n')
#         print(dest_path,'\n')
        movefile(source_path, dest_path)
        
    for file in test_files:
        source_path = temp_dir+path+'/'+file
        dest_path = test_dir+path+'/'+file
        
        movefile(source_path, dest_path)
        
#     print([temp_dir+path+'/'+files for files in test_files],sep='\n')
#     break '/content/drive/My Drive/Colab Notebooks/face recognition/images/temp/pins_Scarlett Johansson/Scarlett Johansson0.jpg'
    

In [2]:
!pip install mtcnn

     |████████████████████████████████| 2.3MB 2.8MB/s 
  Created wheel for mtcnn: filename=mtcnn-0.0.9-cp36-none-any.whl size=2257690 sha256=3b8dbb170895815cb0ab5c8c650e38af96e91895faa231536322171eeace7d35
  Stored in directory: /root/.cache/pip/wheels/85/81/65/6363fa5aafd7a155c896591e0c7c6e27b69642aa82b9cbf076
Successfully built mtcnn


In [3]:
!pip install git+https://github.com/rcmalli/keras-vggface.git

  Cloning https://github.com/rcmalli/keras-vggface.git to /tmp/pip-req-build-t6rdysee
  Running command git clone -q https://github.com/rcmalli/keras-vggface.git /tmp/pip-req-build-t6rdysee
  Created wheel for keras-vggface: filename=keras_vggface-0.6-cp36-none-any.whl size=8311 sha256=75502b9b0b06d5b85558f71bdfdddd5201e09c7268b169a29025b9774dc3b775
  Stored in directory: /tmp/pip-ephem-wheel-cache-p71kia_g/wheels/36/07/46/06c25ce8e9cd396dabe151ea1d8a2bc28dafcb11321c1f3a6d
Successfully built keras-vggface


In [0]:
import numpy as np
import cv2
from keras.models import Model, Sequential, model_from_json
from keras.layers import Input, Convolution2D, ZeroPadding2D, MaxPooling2D, Flatten, Dense, Dropout, Activation
from PIL import Image
from keras.preprocessing.image import load_img, save_img, img_to_array
from keras.applications.imagenet_utils import preprocess_input
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import Adam, RMSprop

from mtcnn.mtcnn import MTCNN

import matplotlib.pyplot as plt

import os
from os import walk
from os import listdir
from os.path import join
import glob

import cv2

In [0]:
count=0
# save each face separately
def save_faces(file, result_list, dir_name, count):
    # load the image
    #data = pyplot.imread(filename)
    #print('file name',file)
    
    #img = cv2.imread(file)
#    print('image name',img)
#     if 'temp folder/test/' in file:
#         file = file.replace('temp folder/test/','test/')
#     if 'temp folder/train/' in file:
#         file = file.replace('temp folder/train/','train/')

    # plot each face as a subplot
    for i in range(len(result_list)):
        # get coordinates
        x1, y1, width, height = result_list[i]['box']
        x2, y2 = x1 + width, y1 + height
        face = file[y1:y2, x1:x2]
        filename = dir_name + str(count) + '.jpg'
        print('file to save',filename)
        cv2.imwrite(filename, face)

In [0]:
def make_dataset(source_folder_list,temp_dir, dest_folder):
    #print(source_folder)
    #print(dest_folder)
    for folder in source_folder_list:
#        print(folder)
        folder_to_save = join(dest_folder,folder)
        print('folder to save ',folder_to_save)
        os.mkdir(folder_to_save)
        for r, d, f in walk(join(temp_dir,folder)):
            #count = 0
            for file in f:
                b = temp_dir+folder+'/'+file
                print('b', b)
                b = os.path.getsize(b)
                if b==0:
                    continue
                image_file = join(join(temp_dir,folder),file)
                print(image_file)
                pixels = plt.imread(image_file)
                if pixels.shape[2]==4:
                    continue
                # detect faces in the image
                faces = detector.detect_faces(pixels)
                # display faces on the original image
                source_file = join(join(temp_dir,folder),file)
                save_faces(source_file, faces, folder_to_save, count)

In [5]:
detector = MTCNN()







Instructions for updating:
Deprecated in favor of operator or tf.math.divide.



In [118]:
temp_test_dir = '/content/drive/My Drive/Colab Notebooks/face recognition/images/temp folder/test/'
temp_train_dir = '/content/drive/My Drive/Colab Notebooks/face recognition/images/temp folder/train/'
temp_train = next(walk(temp_train_dir))[1]


# for _, dir, _ in walk(temp_test_dir):
#     temp_test.append(dir)
# for _, dir, _ in walk(temp_train_dir):
#     temp_train.append(dir)

# print('temp test',temp_test)
#print('temp train', temp_train)
temp_train = next(walk(temp_train_dir))[1]
test = '/content/drive/My Drive/Colab Notebooks/face recognition/images/test/'
train = '/content/drive/My Drive/Colab Notebooks/face recognition/images/train/'



make_dataset(temp_test[1:],temp_test_dir,test)
make_dataset(temp_train[1:],temp_train_dir,train)

# for person in person_dir:
#     for r, d, f in os.walk(temp_train+person):
#         for file in f:
#             print(join(dir,file))
#             pixels = plt.imread(join(dir,file))
#             if pixels.shape[2]==4:
#                 continue
#             # detect faces in the image
#             faces = detector.detect_faces(pixels)
#             # display faces on the original image
#             save_faces(join(dir,file), faces, t_dir, count)

# for (dir,t_dir) in zip(temp_dir,traning_dir):
#     count = 0
#     print('temp dir',dir,'traning dir', t_dir,sep='\n')
#     for r, d, f in walk(dir):
#         for file in f:
#             #print(join(dir,file))
#             pixels = plt.imread(join(dir,file))
#             if pixels.shape[2]==4:
#                 continue
#             # detect faces in the image
#             faces = detector.detect_faces(pixels)
#             # display faces on the original image
#             save_faces(join(dir,file), faces, t_dir, count)

folder to save  /content/drive/My Drive/Colab Notebooks/face recognition/images/train/Willa Holland
b /content/drive/My Drive/Colab Notebooks/face recognition/images/temp folder/train/Willa Holland/Willa Holland0.jpg
/content/drive/My Drive/Colab Notebooks/face recognition/images/temp folder/train/Willa Holland/Willa Holland0.jpg
file to save /content/drive/My Drive/Colab Notebooks/face recognition/images/train/Willa Holland/Willa Holland0.jpg
b /content/drive/My Drive/Colab Notebooks/face recognition/images/temp folder/train/Willa Holland/Willa Holland1.jpg
/content/drive/My Drive/Colab Notebooks/face recognition/images/temp folder/train/Willa Holland/Willa Holland1.jpg
file to save /content/drive/My Drive/Colab Notebooks/face recognition/images/train/Willa Holland/Willa Holland1.jpg
b /content/drive/My Drive/Colab Notebooks/face recognition/images/temp folder/train/Willa Holland/Willa Holland10.jpg
/content/drive/My Drive/Colab Notebooks/face recognition/images/temp folder/train/Will

In [0]:
path = '/content/drive/My Drive/Colab Notebooks/face recognition/20191014_180017.mp4'
vidObj = cv2.VideoCapture(path) 
  
# Used as counter variable 
count = 0

# checks whether frames were extracted 
success = 1

while success: 

    # vidObj object calls read 
    # function extract frames 
    success, image = vidObj.read() 

    # Saves the frames with frame-count 
    cv2.imwrite("/content/drive/My Drive/Colab Notebooks/face recognition/images/temp/my face/frame%d.jpg" % count, image) 

    count += 1

In [0]:
def rotateImage(image, angle):
  image_center = tuple(np.array(image.shape[1::-1]) / 2)
  rot_mat = cv2.getRotationMatrix2D(image_center, angle, 1.0)
  result = cv2.warpAffine(image, rot_mat, image.shape[1::-1], flags=cv2.INTER_LINEAR)
  return result

In [159]:
path = '/content/drive/My Drive/Colab Notebooks/face recognition/images/temp/my face/'
#path = '/content/drive/My Drive/Colab Notebooks/face recognition/images/temp folder/train/Scarlett Johansson'
files = next(walk(path))[2]
#print(len(files))
#print(files[0:5])
count=0
for i,file in enumerate(files):
    if i%3==0:
        img = cv2.imread(join(path,file))
        img = rotateImage(img,90)
        #plt.imshow(img)
        #break
        faces = detector.detect_faces(img)
        source_file = join(path,file)
        folder_to_save = '/content/drive/My Drive/Colab Notebooks/face recognition/images/temp folder/train/kaushal/'
        save_faces(img, faces, folder_to_save, count)
        count+=1
    

file to save /content/drive/My Drive/Colab Notebooks/face recognition/images/temp folder/train/kaushal/0.jpg
file to save /content/drive/My Drive/Colab Notebooks/face recognition/images/temp folder/train/kaushal/1.jpg
file to save /content/drive/My Drive/Colab Notebooks/face recognition/images/temp folder/train/kaushal/2.jpg
file to save /content/drive/My Drive/Colab Notebooks/face recognition/images/temp folder/train/kaushal/3.jpg
file to save /content/drive/My Drive/Colab Notebooks/face recognition/images/temp folder/train/kaushal/4.jpg
file to save /content/drive/My Drive/Colab Notebooks/face recognition/images/temp folder/train/kaushal/5.jpg
file to save /content/drive/My Drive/Colab Notebooks/face recognition/images/temp folder/train/kaushal/6.jpg
file to save /content/drive/My Drive/Colab Notebooks/face recognition/images/temp folder/train/kaushal/8.jpg
file to save /content/drive/My Drive/Colab Notebooks/face recognition/images/temp folder/train/kaushal/9.jpg
file to save /conte

AttributeError: ignored

In [167]:
path = '/content/drive/My Drive/Colab Notebooks/face recognition/images/temp folder/train/kaushal'
files = next(walk(path))[2]
dest = '/content/drive/My Drive/Colab Notebooks/face recognition/images/temp folder/test/kaushal'
for i,file in enumerate(files):
    print(file)
    if i%5==0:
        source = join(path,file)
        dest_file = join(dest,file)
        os.rename(source,dest_file)
        print('source',source)
        print('dest',dest)

4.jpg
source /content/drive/My Drive/Colab Notebooks/face recognition/images/temp folder/train/kaushal/4.jpg
dest /content/drive/My Drive/Colab Notebooks/face recognition/images/temp folder/test/kaushal
5.jpg
6.jpg
8.jpg
9.jpg
10.jpg
source /content/drive/My Drive/Colab Notebooks/face recognition/images/temp folder/train/kaushal/10.jpg
dest /content/drive/My Drive/Colab Notebooks/face recognition/images/temp folder/test/kaushal
11.jpg
12.jpg
13.jpg
14.jpg
15.jpg
source /content/drive/My Drive/Colab Notebooks/face recognition/images/temp folder/train/kaushal/15.jpg
dest /content/drive/My Drive/Colab Notebooks/face recognition/images/temp folder/test/kaushal
16.jpg
17.jpg
18.jpg
19.jpg
20.jpg
source /content/drive/My Drive/Colab Notebooks/face recognition/images/temp folder/train/kaushal/20.jpg
dest /content/drive/My Drive/Colab Notebooks/face recognition/images/temp folder/test/kaushal
0.jpg
1.jpg
2.jpg
21.jpg
22.jpg
source /content/drive/My Drive/Colab Notebooks/face recognition/images

In [0]:
test_dirts = ['/content/drive/My Drive/Colab Notebooks/face recognition/test/Scarlett Johansson','/content/drive/My Drive/Colab Notebooks/face recognition/test/Willa Holland','/content/drive/My Drive/Colab Notebooks/face recognition/test/alexandra daddario','/content/drive/My Drive/Colab Notebooks/face recognition/test/amber heard face','/content/drive/My Drive/Colab Notebooks/face recognition/test/anne hathaway']
dest_dir = ['/content/drive/My Drive/Colab Notebooks/face recognition/images/temp folder/test/Scarlett Johansson','/content/drive/My Drive/Colab Notebooks/face recognition/images/temp folder/test/Willa Holland','/content/drive/My Drive/Colab Notebooks/face recognition/images/temp folder/test/alexandra daddario','/content/drive/My Drive/Colab Notebooks/face recognition/images/temp folder/test/amber heard face','/content/drive/My Drive/Colab Notebooks/face recognition/images/temp folder/test/anne hathaway']
count = -1
for dir in test_dirts:
    for r, d,f in walk(dir):
        #print(r)
        count+=1
        for file in f:
            pixels = cv2.imread(join(r,file))
            dest = dest_dir[count]
            
            #print(join(r,file))
            result_list = detector.detect_faces(pixels)
            # display faces on the original image
            for i in range(len(result_list)):
                # get coordinates
                x1, y1, width, height = result_list[i]['box']
                x2, y2 = x1 + width, y1 + height
                face = pixels[y1:y2, x1:x2]
                #filename = dir_name + str(count) + '.jpg'
                #print(join(dest,file))
                cv2.imwrite(join(dest,file), face)
                #save_faces(join(dir,file), faces, t_dir, count)

In [0]:
batch_size = 20
epochs = 2
steps_per_epoch = 100

In [8]:
train_dir = '/content/drive/My Drive/Colab Notebooks/face recognition/images/temp folder/train'
test_dir = '/content/drive/My Drive/Colab Notebooks/face recognition/images/temp folder/test'

input_shape=(224,224)
datagen_train = ImageDataGenerator(
      rescale=1./255,
      rotation_range=45,
      vertical_flip=True,
      fill_mode='nearest')
datagen_test = ImageDataGenerator(rescale=1./255)
generator_train = datagen_train.flow_from_directory(directory=train_dir,
                                                    target_size=input_shape,
                                                    batch_size=batch_size,
                                                    shuffle=True)
generator_test = datagen_test.flow_from_directory(directory=test_dir,
                                                  target_size=input_shape,
                                                  batch_size=batch_size,
                                                  shuffle=False)

steps_test = generator_test.n / batch_size
num_classes = generator_train.num_classes
cls_train = generator_train.classes
cls_test = generator_test.classes

Found 487 images belonging to 6 classes.
Found 147 images belonging to 6 classes.


In [9]:
print(steps_test,num_classes,cls_train,cls_test,sep='\n')

7.35
6
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3
 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3
 3 3 3 3 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4
 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4
 4 4 4 4 4 4 4 4 4 4 4 4 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5
 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5
 5 5 5 5 5 5]
[0 0

In [0]:
def path_join(dirname, filenames):
    return [os.path.join(dirname, filename) for filename in filenames]

In [13]:
steps_test = generator_test.n / batch_size
image_paths_train = path_join(train_dir, generator_train.filenames)
image_paths_test = path_join(test_dir, generator_test.filenames)
cls_train = generator_train.classes
cls_test = generator_test.classes
class_names = list(generator_train.class_indices.keys())
print('class name',class_names)
num_classes = generator_train.num_classes
print('num classes',num_classes)

class name ['Scarlett Johansson', 'Willa Holland', 'alexandra daddario', 'amber heard face', 'anne hathaway', 'kaushal']
num classes 6


In [14]:
def loadVggFaceModel():
    model = Sequential()
    model.add(ZeroPadding2D((1,1),input_shape=(224,224, 3)))
    model.add(Convolution2D(64, (3, 3), activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(64, (3, 3), activation='relu'))
    model.add(MaxPooling2D((2,2), strides=(2,2)))

    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(128, (3, 3), activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(128, (3, 3), activation='relu'))
    model.add(MaxPooling2D((2,2), strides=(2,2)))

    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(256, (3, 3), activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(256, (3, 3), activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(256, (3, 3), activation='relu'))
    model.add(MaxPooling2D((2,2), strides=(2,2)))

    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(512, (3, 3), activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(512, (3, 3), activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(512, (3, 3), activation='relu'))
    model.add(MaxPooling2D((2,2), strides=(2,2)))

    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(512, (3, 3), activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(512, (3, 3), activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(512, (3, 3), activation='relu'))
    model.add(MaxPooling2D((2,2), strides=(2,2)))

    model.add(Convolution2D(4096, (7, 7), activation='relu'))
    model.add(Dropout(0.5))
    model.add(Convolution2D(4096, (1, 1), activation='relu'))
    model.add(Dropout(0.5))
    model.add(Convolution2D(2622, (1, 1)))
    model.add(Flatten())
    model.add(Activation('softmax'))
    model.load_weights('/content/drive/My Drive/Colab Notebooks/face recognition/vgg_face_weights.h5')
    loaded_model = Model(inputs=model.layers[0].input, outputs=model.layers[-2].output)
#     predictions = Dense(6, activation = 'softmax')(loaded_model.output)
#     new_model = Model(input = loaded_model.input, output = predictions)
    return loaded_model

vgg_model = loadVggFaceModel()
vgg_model.summary()




Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.




Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
zero_padding2d_1_input (Inpu (None, 224, 224, 3)       0         
_________________________________________________________________
zero_padding2d_1 (ZeroPaddin (None, 226, 226, 3)       0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 224, 224, 64)      1792      
_________________________________________________________________
zero_padding2d_2 (ZeroPaddin (None, 226, 226, 64)      0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 224, 224, 64)      36928     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 112, 112, 64)      0         
___

In [0]:
new_model = Sequential()
new_model.add(vgg_model)
new_model.add(Dense(1024, activation='relu'))
new_model.add(Dropout(.5))
new_model.add(Dense(num_classes,activation='softmax'))

In [0]:
optimizer = Adam(lr=1e-5)
loss = 'categorical_crossentropy'
metrics = ['categorical_accuracy']

In [17]:
def print_layer_trainable(model):
    for layer in model.layers:
        print("{0}:\t{1}".format(layer.trainable, layer.name))
        
print_layer_trainable(vgg_model)

False:	zero_padding2d_1_input
True:	zero_padding2d_1
True:	conv2d_1
True:	zero_padding2d_2
True:	conv2d_2
True:	max_pooling2d_1
True:	zero_padding2d_3
True:	conv2d_3
True:	zero_padding2d_4
True:	conv2d_4
True:	max_pooling2d_2
True:	zero_padding2d_5
True:	conv2d_5
True:	zero_padding2d_6
True:	conv2d_6
True:	zero_padding2d_7
True:	conv2d_7
True:	max_pooling2d_3
True:	zero_padding2d_8
True:	conv2d_8
True:	zero_padding2d_9
True:	conv2d_9
True:	zero_padding2d_10
True:	conv2d_10
True:	max_pooling2d_4
True:	zero_padding2d_11
True:	conv2d_11
True:	zero_padding2d_12
True:	conv2d_12
True:	zero_padding2d_13
True:	conv2d_13
True:	max_pooling2d_5
True:	conv2d_14
True:	dropout_1
True:	conv2d_15
True:	dropout_2
True:	conv2d_16
True:	flatten_1


In [18]:
vgg_model.trainable = False #i dont know why it didnt work
for layer in vgg_model.layers:
    layer.trainable = False
print_layer_trainable(new_model)
print('new model')
print_layer_trainable(vgg_model)

False:	model_1
True:	dense_1
True:	dropout_3
True:	dense_2
new model
False:	zero_padding2d_1_input
False:	zero_padding2d_1
False:	conv2d_1
False:	zero_padding2d_2
False:	conv2d_2
False:	max_pooling2d_1
False:	zero_padding2d_3
False:	conv2d_3
False:	zero_padding2d_4
False:	conv2d_4
False:	max_pooling2d_2
False:	zero_padding2d_5
False:	conv2d_5
False:	zero_padding2d_6
False:	conv2d_6
False:	zero_padding2d_7
False:	conv2d_7
False:	max_pooling2d_3
False:	zero_padding2d_8
False:	conv2d_8
False:	zero_padding2d_9
False:	conv2d_9
False:	zero_padding2d_10
False:	conv2d_10
False:	max_pooling2d_4
False:	zero_padding2d_11
False:	conv2d_11
False:	zero_padding2d_12
False:	conv2d_12
False:	zero_padding2d_13
False:	conv2d_13
False:	max_pooling2d_5
False:	conv2d_14
False:	dropout_1
False:	conv2d_15
False:	dropout_2
False:	conv2d_16
False:	flatten_1


In [19]:
new_model.compile(optimizer=optimizer, loss=loss, metrics=metrics)

In [21]:
from sklearn.utils.class_weight import compute_class_weight
class_weight = compute_class_weight(class_weight='balanced',
                                    classes=np.unique(cls_train),
                                    y=cls_train)
print(class_weight)

[0.86347518 1.22979798 0.75154321 1.1763285  0.9898374  1.19362745]


In [23]:
#fit new vgg model
all_history =[]
while 1:
    history = new_model.fit_generator(
            generator=generator_train,
            epochs=epochs,
            steps_per_epoch=steps_per_epoch,
            class_weight=class_weight,
            validation_data=generator_test)
    all_history.append(history)
    generator_test.reset()
    result = new_model.evaluate_generator(generator_test, steps=steps_test)
    print('result: ',result)
    if result[1]>.8:
        break

Epoch 1/2
100/100 [==============================] - 42s 424ms/step - loss: 1.7109 - categorical_accuracy: 0.4133 - val_loss: 1.6414 - val_categorical_accuracy: 0.5510
Epoch 2/2
100/100 [==============================] - 41s 414ms/step - loss: 1.6780 - categorical_accuracy: 0.4316 - val_loss: 1.5903 - val_categorical_accuracy: 0.5510
result:  [1.5903023359726887, 0.5510204009661058]
Epoch 1/2
100/100 [==============================] - 43s 433ms/step - loss: 1.6402 - categorical_accuracy: 0.4430 - val_loss: 1.5375 - val_categorical_accuracy: 0.5714
Epoch 2/2
100/100 [==============================] - 42s 418ms/step - loss: 1.6045 - categorical_accuracy: 0.4714 - val_loss: 1.4860 - val_categorical_accuracy: 0.5918
result:  [1.4859922216052102, 0.5918367346938775]
Epoch 1/2
100/100 [==============================] - 43s 434ms/step - loss: 1.5812 - categorical_accuracy: 0.4743 - val_loss: 1.4356 - val_categorical_accuracy: 0.6122
Epoch 2/2
100/100 [==============================] - 41s 412

In [24]:
#saving and loading model from transfer learning
model_json = new_model.to_json()
with open("./transfer_model.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
new_model.save_weights("./transfer_model.h5")
print("Saved model to disk")

# load json and create model
json_file = open('./transfer_model.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_vgg_model = model_from_json(loaded_model_json)
# load weights into new model
loaded_vgg_model.load_weights("./transfer_model.h5")
print("Loaded model from disk")

Saved model to disk
Loaded model from disk


In [27]:
loaded_vgg_model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
model_1 (Model)              (None, 2622)              145002878 
_________________________________________________________________
dense_1 (Dense)              (None, 1024)              2685952   
_________________________________________________________________
dropout_3 (Dropout)          (None, 1024)              0         
_________________________________________________________________
dense_2 (Dense)              (None, 6)                 6150      
Total params: 147,694,980
Trainable params: 2,692,102
Non-trainable params: 145,002,878
_________________________________________________________________


In [28]:
print_layer_trainable(loaded_vgg_model)
for layer in loaded_vgg_model.layers:
    layer.trainable = False
for layer in loaded_vgg_model.layers[-17:]:
    layer.trainable = True
print('after unfreazing')
print_layer_trainable(loaded_vgg_model)

True:	model_1
True:	dense_1
True:	dropout_3
True:	dense_2
after unfreazing
True:	model_1
True:	dense_1
True:	dropout_3
True:	dense_2


In [0]:
optimizer = Adam(lr=1e-5)
loss = 'categorical_crossentropy'
metrics = ['categorical_accuracy']
loaded_vgg_model.compile(optimizer=optimizer, loss=loss, metrics=metrics)

In [0]:
all_history =[]
count =0
while 1:
    history=loaded_vgg_model.fit_generator(
            generator=generator_train,
            epochs=epochs,
            steps_per_epoch=steps_per_epoch,
            class_weight=class_weight,
            validation_data=generator_test)
    all_history.append(history)
    generator_test.reset()
    result = loaded_vgg_model.evaluate_generator(generator_test, steps=steps_test)
    print('result: ',result)
    if result[1]>.98:
        break

Epoch 1/2
100/100 [==============================] - 42s 425ms/step - loss: 1.3962 - categorical_accuracy: 0.5651 - val_loss: 1.1499 - val_categorical_accuracy: 0.8163
Epoch 2/2
100/100 [==============================] - 40s 403ms/step - loss: 1.3850 - categorical_accuracy: 0.5718 - val_loss: 1.1105 - val_categorical_accuracy: 0.8231
result:  [1.1104823237373715, 0.8231292500787851]
Epoch 1/2
100/100 [==============================] - 43s 427ms/step - loss: 1.3533 - categorical_accuracy: 0.5950 - val_loss: 1.0729 - val_categorical_accuracy: 0.8299
Epoch 2/2
100/100 [==============================] - 40s 404ms/step - loss: 1.3327 - categorical_accuracy: 0.5978 - val_loss: 1.0350 - val_categorical_accuracy: 0.8503
result:  [1.034967992176004, 0.8503401328106316]
Epoch 1/2
100/100 [==============================] - 42s 421ms/step - loss: 1.3248 - categorical_accuracy: 0.5904 - val_loss: 0.9991 - val_categorical_accuracy: 0.8639
Epoch 2/2
100/100 [==============================] - 41s 406m

In [0]:
#saving and loading fine tuned model
model_json = loaded_vgg_model.to_json()
with open("./finTuned_model.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
loaded_vgg_model.save_weights("./finTuned_model.h5")
print("Saved model to disk")

# load json and create model
json_file = open('./finTuned_model.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_vgg_model = model_from_json(loaded_model_json)
# load weights into new model
loaded_vgg_model.load_weights("./finTuned_model.h5")
print("Loaded model from disk")

In [0]:
optimizer = Adam(lr=1e-5)
loss = 'categorical_crossentropy'
metrics = ['categorical_accuracy']
loaded_vgg_model.compile(optimizer=optimizer, loss=loss, metrics=metrics)

In [0]:
cap = cv2.VideoCapture(0) 

while(True):
	ret, img = cap.read()
	#img = cv2.resize(img, (640, 360))
	#faces = face_cascade.detectMultiScale(img, 1.3, 5)
    faces = detector.detect_faces(img)
	
	for (x,y,w,h) in faces:
		if w > 130: 
			#cv2.rectangle(img,(x,y),(x+w,y+h),(255,0,0),2) #draw rectangle to main image
			
			detected_face = img[int(y):int(y+h), int(x):int(x+w)] #crop detected face
			detected_face = cv2.resize(detected_face, (224, 224)) #resize to 224x224
			
			img_pixels = image.img_to_array(detected_face)
			img_pixels = np.expand_dims(img_pixels, axis = 0)
			
			img_pixels /= 127.5
			img_pixels -= 1
			
			captured_representation = loaded_vgg_model.predict(img_pixels)
            
            print('captured_representation ',captured_representation)
            predicted_person = captured_representation.index(max(captured_representation))
            print('predicted_person ',predicted_person)
            confidence = max(captured_representation)
            print('confidence ',confidence)
            #y_prob = model.predict(x) 
            y_classes = captured_representation.argmax(axis=-1)
            print(y_classes)
            #text = class_names[predicted_person]
            
            if confidence>.6:
                text = class_names[predicted_person]
            else:
                text = 'unknown'
            cv2.putText(img, text, (int(x+w+15), int(y-12)), cv2.FONT_HERSHEY_SIMPLEX, 1, color, 2)
			
# 			found = 0
# 			for i in employees:
# 				employee_name = i
# 				representation = employees[i]
				
# 				similarity = findCosineSimilarity(representation, captured_representation)
# 				if(similarity < 0.30):
# 					cv2.putText(img, employee_name, (int(x+w+15), int(y-12)), cv2.FONT_HERSHEY_SIMPLEX, 1, color, 2)
					
# 					found = 1
# 					break
					
			#connect face and text
			cv2.line(img,(int((x+x+w)/2),y+15),(x+w,y-20),color,1)
			cv2.line(img,(x+w,y-20),(x+w+10,y-20),color,1)
		
# 			if(found == 0): #if found image is not in employee database
# 				cv2.putText(img, 'unknown', (int(x+w+15), int(y-12)), cv2.FONT_HERSHEY_SIMPLEX, 1, color, 2)
#             else:
#                 cv2.putText(img, person_name, (int(x+w+15), int(y-12)), cv2.FONT_HERSHEY_SIMPLEX, 1, color, 2)
	
	cv2.imshow('img',img)
	
	if cv2.waitKey(1) & 0xFF == ord('q'): #press q to quit
		break
	
#kill open cv things		
cap.release()
cv2.destroyAllWindows()